In [1]:
# import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = '\
# --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.2.0,com.datastax.spark:spark-cassandra-connector_2.11:2.0.1 \
# pyspark-shell'

In [2]:
# #    Spark
# from pyspark import SparkConf, SparkContext
# #    Spark Streaming
# from pyspark.streaming import StreamingContext
# #    Kafka
# from pyspark.streaming.kafka import KafkaUtils

# from pyspark.sql import SQLContext

# #    json parsing
# import json

In [3]:
# conf = SparkConf(True).setMaster("local[*]").setAppName("jupyter pyspark").set("spark.cassandra.connection.host", "cassandra")
# sc = SparkContext(conf=conf)
# #sc = SparkContext(appName= "jupyter pyspark")

# sc.setLogLevel("WARN")
# sqlContext = SQLContext(sc)

In [4]:
#sc.stop()

In [15]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import pandas.tseries.offsets as offsets
%matplotlib inline

In [6]:
sf_data = pd.read_csv("./raw_data/sanfrancisco.csv")

In [7]:
raw_columns = list(sf_data)
raw_columns

['dt',
 'dt_iso',
 'city_id',
 'city_name',
 'lat',
 'lon',
 'temp',
 'temp_min',
 'temp_max',
 'pressure',
 'sea_level',
 'grnd_level',
 'humidity',
 'wind_speed',
 'wind_deg',
 'rain_1h',
 'rain_3h',
 'rain_24h',
 'rain_today',
 'snow_1h',
 'snow_3h',
 'snow_24h',
 'snow_today',
 'clouds_all',
 'weather_id',
 'weather_main',
 'weather_description',
 'weather_icon']

In [8]:
unused_columns = ['snow_1h', 'snow_24h', 'rain_24h', 'rain_1h', 'snow_3h', 'rain_today', 'snow_today', 'weather_icon', 'weather_id', 'sea_level', 'grnd_level', 'lat', 'lon', 'city_id', 'city_name']

In [9]:
columns = list(set(raw_columns) - set(unused_columns))

In [10]:
sf_data2 = sf_data[columns]

In [11]:
sf_data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38415 entries, 0 to 38414
Data columns (total 14 columns):
weather_id             38415 non-null int64
wind_speed             38415 non-null int64
dt                     38415 non-null int64
rain_3h                1355 non-null float64
temp_min               38415 non-null float64
wind_deg               38415 non-null int64
clouds_all             38415 non-null int64
dt_iso                 38415 non-null object
temp_max               38415 non-null float64
pressure               38415 non-null int64
humidity               38415 non-null int64
weather_main           38415 non-null object
temp                   38415 non-null float64
weather_description    38415 non-null object
dtypes: float64(4), int64(7), object(3)
memory usage: 4.1+ MB


In [12]:
sf_data2['rain_3h']= sf_data2['rain_3h'].fillna(0)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
sf_data2.drop_duplicates('dt', inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/util/decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


In [26]:
sf_data2['dt_datetime'] =  pd.to_datetime(sf_data2['dt_iso'], format='%Y-%m-%d %H:%M:%S +%f %Z')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [141]:
sf_data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33442 entries, 0 to 38414
Data columns (total 15 columns):
weather_id             33442 non-null int64
wind_speed             33442 non-null int64
dt                     33442 non-null int64
rain_3h                33442 non-null float64
temp_min               33442 non-null float64
wind_deg               33442 non-null int64
clouds_all             33442 non-null int64
dt_iso                 33442 non-null object
temp_max               33442 non-null float64
pressure               33442 non-null int64
humidity               33442 non-null int64
weather_main           33442 non-null object
temp                   33442 non-null float64
weather_description    33442 non-null object
dt_datetime            33442 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(7), object(3)
memory usage: 4.1+ MB


In [42]:
int(pd.Timedelta(sf_data2['dt_datetime'].iloc[2] - sf_data2['dt_datetime'].iloc[1]).seconds/3600)

1

In [187]:
diff_columns = list(set(sf_data2.columns) - set(['dt', 'dt_iso']))
diff_columns
day_diff = 2
maximum_prev_index_diff = 2 *  24 
raw_datum = sf_data2.reset_index(drop=True).iloc[0:100]
raw_datum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 15 columns):
weather_id             100 non-null int64
wind_speed             100 non-null int64
dt                     100 non-null int64
rain_3h                100 non-null float64
temp_min               100 non-null float64
wind_deg               100 non-null int64
clouds_all             100 non-null int64
dt_iso                 100 non-null object
temp_max               100 non-null float64
pressure               100 non-null int64
humidity               100 non-null int64
weather_main           100 non-null object
temp                   100 non-null float64
weather_description    100 non-null object
dt_datetime            100 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(7), object(3)
memory usage: 11.8+ KB


In [188]:
def add_diff_datum():
    data = raw_datum.copy()
    data = data.apply(add_diff_days_data, axis=1)
    return data
sf_data3 = add_diff_datum()

In [183]:
def add_diff_days_data(current_data):    
    index = current_data.name

    if index == 0:
        return current_data
    prev_index_right = index - 1
    left_index = index - maximum_prev_index_diff if index - maximum_prev_index_diff > 0 else 0
       
    while left_index <= prev_index_right:
        current_data = add_diff_data(current_data, raw_datum.iloc[left_index])
        left_index += 1
    return current_data
    

In [159]:
def add_diff_data(current_data, prev_data):
    diff = int(pd.Timedelta(current_data['dt_datetime'] - prev_data['dt_datetime']).seconds/ 3600)
    if diff > 0 and diff < maximum_prev_index_diff:
        for diff_column in diff_columns:
            column_name = '{}_{}_ago'.format(diff_column, diff)
            current_data[column_name] = prev_data[diff_column]
    return current_data

In [215]:
[1,2] + [3]

[1, 2, 3]

In [ ]:
#create 2 d diff data
# change category data to numercial dataa
# explore data
# apply machine learning algorithm

In [225]:
def convert_category_to_numercial_data(raw_data):
    weather_description_columns = list(raw_data['weather_description'])
    weather_main_columns = list(raw_data['weather_main'])
    
    data = raw_data.copy()
    data = data.apply(transform_categorical_data, args=(weather_description_columns, weather_main_columns), axis=1)
    data = data.apply(transform_datetime, axis=1)
    
    
    new_data = data.drop(unused_columns, axis=1)
    return new_data
    
def transform_datetime(current_data):
    for month in range(1, 12):
        current_data['month_{}'.format(month)] = 1 if current_data['dt_datetime'].month == month else 0
       
    current_data['year'] =  current_data['dt_datetime'].year
    current_data['dayofweek'] = current_data['dt_datetime'].dayofweek
    current_data['dayofyear'] = current_data['dt_datetime'].dayofyear
    current_data['hourofday'] = current_data['dt_datetime'].hour
    return current_data

def transform_categorical_data(current_data,weather_description_columns, weather_main_columns):
    
    for column in weather_description_columns:
   
        current_data[column] = 1 if current_data['weather_description'] == column else 0
    for column in weather_main_columns:
        current_data[column] = 1 if current_data['weather_main'] == column else 0
    return current_data
    

            
            
    

In [226]:
sf_data2_b = convert_category_to_numercial_data(sf_data2.iloc[0:100])
sf_data2_b

,wind_speed,dt,rain_3h,temp_min,wind_deg,clouds_all,temp_max,pressure,humidity,temp,...,month_6,month_7,month_8,month_9,month_10,month_11,year,dayofweek,dayofyear,hourofday
0,2,1349096400,0.0,282.15,150,0,301.15,1009,88,289.48,...,0,0,0,0,1,0,2012,0,275,13
2,0,1349186400,0.0,282.15,0,1,301.15,1015,63,289.13,...,0,0,0,0,1,0,2012,1,276,14
3,0,1349190000,0.0,283.15,0,1,302.15,1016,51,290.73,...,0,0,0,0,1,0,2012,1,276,15
4,4,1349193600,0.0,287.15,190,0,307.15,1011,56,293.02,...,0,0,0,0,1,0,2012,1,276,16
5,3,1349197200,0.0,290.15,140,0,307.15,1001,94,296.18,...,0,0,0,0,1,0,2012,1,276,17
6,4,1349200800,0.0,293.71,230,40,305.93,1009,70,299.66,...,0,0,0,0,1,0,2012,1,276,18
7,6,1349204400,0.0,293.15,170,90,308.15,1000,83,300.03,...,0,0,0,0,1,0,2012,1,276,19
8,3,1349208000,0.0,293.15,250,0,308.71,1007,75,301.07,...,0,0,0,0,1,0,2012,1,276,20
9,3,1349211600,0.0,295.15,260,75,308.71,1007,75,302.29,...,0,0,0,0,1,0,2012,1,276,21
10,6,1349215200,0.0,296.15,300,1,309.26,1013,18,304.70,...,0,0,0,0,1,0,2012,1,276,22


In [227]:
sf_data2_b.columns

Index(['wind_speed', 'dt', 'rain_3h', 'temp_min', 'wind_deg', 'clouds_all',
       'temp_max', 'pressure', 'humidity', 'temp', 'light intensity drizzle',
       'sky is clear', 'haze', 'scattered clouds', 'overcast clouds',
       'broken clouds', 'mist', 'few clouds', 'fog', 'Drizzle', 'Clear',
       'Haze', 'Clouds', 'Mist', 'Fog', 'month_1', 'month_2', 'month_3',
       'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9',
       'month_10', 'month_11', 'year', 'dayofweek', 'dayofyear', 'hourofday'],
      dtype='object')

In [ ]:
def 